In [6]:
# Importar las librerias necesarias
import pandas as pd

In [7]:
# Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.
data = pd.read_csv("dataset_pishing.csv")

# Hacemos el describe para ver la distribución de los datos
print(data.describe())

# Hacemos el head para ver las primeras 5 filas
print(data.head())

                                                      url      status
count                                               11430       11430
unique                                              11429           2
top     http://e710z0ear.du.r.appspot.com/c:/users/use...  legitimate
freq                                                    2        5715
                                                 url      status
0              http://www.crestonwood.com/router.php  legitimate
1  http://shadetreetechnology.com/V4/validation/a...    phishing
2  https://support-appleld.com.secureupdate.duila...    phishing
3                                 http://rgipt.ac.in  legitimate
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate


In [8]:
# Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset?

print(data["status"].value_counts())

status
legitimate    5715
phishing      5715
Name: count, dtype: int64


### Derivación de características 
En base a los artículos propuestos de clasificación de phishing, responda las siguientes preguntas: 
1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo 
de vida del dominio, o las características de la página Web? 
2. ¿Qué características de una URL son más prometedoras para la detección de phishing?

### 1. Ventajas del Análisis de URL contra Otros Datos
Eficiencia y Accesibilidad: El análisis de URL permite una evaluación rápida y en tiempo real sin cargar la página, facilitando la detección preliminar de amenazas de forma eficiente.
Consistencia de Datos: Las URLs proporcionan información estática y directa, menos susceptible a cambios y manipulaciones, en contraste con el contenido dinámico de las páginas web que puede ser más fácilmente alterado para evadir la detección.

### 2. Características de una URL más Prometedoras para la Detección de Phishing
Dominios Mal Escritos y Uso de Subdominios: Las variaciones ortográficas sutiles en dominios y el uso excesivo de subdominios intentan imitar sitios legítimos para engañar a los usuarios, siendo indicativos clave de phishing.
Longitud de la URL y Presencia de Palabras Clave Sensibles: Las URLs anormalmente largas y la inclusión de términos relacionados con la seguridad o transacciones financieras pueden señalar intentos de phishing, buscando simular urgencia o legitimidad.

IndexError: list index out of range